# Exploitation des déséquilibres Polymarket sur BTC

Ce carnet assemble la chaîne complète : chargement des données, calibration d'un modèle de cotes FOMO, estimation de la probabilité réelle de clôture d'une bougie et backtests des stratégies 2 % capital et 4 % de parts.


## Plan du carnet

1. Chargement et préparation des données (1s et 1m)
2. Simulation de cotes FOMO et visualisations
3. Entraînement du modèle de cotes vs marché
4. Entraînement du modèle de probabilité réelle
5. Génération des cotes synthétiques historiques et backtests


### Préparation des dépendances

**Objectif** : charger toutes les fonctions utilitaires définies dans `notebooks/code` nécessaires au pipeline complet.

**Sorties attendues** : aucune sortie directe. Les imports réussis garantissent que chaque module est disponible pour les cellules suivantes. En cas d'erreur, vérifier que les fichiers sont bien présents et que les dépendances Python sont installées.


In [ ]:
from pathlib import Path

import pandas as pd

from code.pipeline import (
    load_all_data,
    prepare_ohlc_features,
    enrich_polymarket_with_features,
    prepare_timeframe_tables,
    build_regression_dataset,
    build_classification_dataset,
    prepare_minute_history,
    select_recent_rows,
    make_fomo_input,
    estimate_average_spreads,
)
from code.fomo_simulation import make_default_scenarios, simulate_fomo_odds
from code.model_training import (
    train_odds_regressors,
    train_outcome_classifiers,
    predict_regressions,
    predict_classifications,
)
from code.backtest import BacktestParams, run_backtest
from code.visualization import (
    plot_odds_comparison,
    plot_equity_curves,
    plot_calibration_curve,
)
from code.persistence import (
    export_regression_artifacts,
    export_classification_artifacts,
)
from code.pricing import probabilities_to_prices



### Chargement et enrichissement des données

**Objectif** : lire les flux Polymarket (1 seconde) et les bougies BTC (1 minute), construire les features techniques, préparer les tables par horizon (m15, h1, daily) et stocker les structures intermédiaires utiles aux modèles.

**Sorties attendues** : affichage des premières lignes de `polymarket_raw` et `ohlc_raw` pour valider le chargement. Des dictionnaires `regression_data`, `classification_data`, `fomo_inputs` ainsi que `minute_history` sont créés en mémoire pour une réutilisation par les cellules suivantes. Si certains champs sont `NaN`, vérifier la cohérence des horodatages ou ajuster `select_recent_rows`.


In [ ]:
polymarket_raw, ohlc_raw = load_all_data()
polymarket_subset = select_recent_rows(polymarket_raw, 500_000)
ohlc_features = prepare_ohlc_features(ohlc_raw)
per_second_base = enrich_polymarket_with_features(polymarket_subset, ohlc_features)

timeframes = ["m15", "h1", "daily"]
per_second_full = per_second_base.copy()
for tf in timeframes:
    per_second_full = prepare_timeframe_tables(per_second_full, tf)

average_spreads = estimate_average_spreads(per_second_full, timeframes)

timeframe_tables = {}
regression_data = {}
classification_data = {}
fomo_inputs = {}
for tf in timeframes:
    tf_df = prepare_timeframe_tables(per_second_base.copy(), tf)
    timeframe_tables[tf] = tf_df
    reg_dataset, reg_features, reg_targets = build_regression_dataset(tf_df, tf)
    regression_data[tf] = {
        "dataset": reg_dataset,
        "features": reg_features,
        "targets": reg_targets,
    }
    cls_dataset, cls_features, cls_target = build_classification_dataset(tf_df, tf)
    classification_data[tf] = {
        "dataset": cls_dataset,
        "features": cls_features,
        "target": cls_target,
    }
    fomo_inputs[tf] = make_fomo_input(tf_df, tf)

minute_history = {tf: prepare_minute_history(ohlc_features, tf) for tf in timeframes}

polymarket_raw.head(), ohlc_raw.head()


## Simulation FOMO calibrée sur les données réelles

**Objectif** : générer des cotes théoriques à partir de scénarios FOMO (agressif, modéré, conservateur) en exploitant les probabilités et ATR issus des données historiques.

**Sorties attendues** : les tables simulées contiendront pour chaque contrat une colonne `prob_up` (référence marché) et plusieurs colonnes `odds_*` représentant les variantes FOMO. Les estimations initiales de cote sont imprimées pour trois contrats par horizon afin de valider l’ancrage des simulations.

In [ ]:
fomo_scenarios = {tf: make_default_scenarios(tf) for tf in timeframes}
fomo_results = {}
for tf in timeframes:
    fomo_results[tf] = simulate_fomo_odds(fomo_inputs[tf], fomo_scenarios[tf])

initial_quotes = {
    tf: (
        fomo_results[tf].groupby("contract_id").head(1)[
            ["timestamp", "prob_up"] + [col for col in fomo_results[tf].columns if col.startswith("odds_")]
        ].head(3)
    )
    for tf in timeframes
}

initial_quotes


In [ ]:
for tf in timeframes:
    scenario_cols = [col for col in fomo_results[tf].columns if col.startswith("odds_")][:3]
    figure = plot_odds_comparison(fomo_results[tf], tf, scenario_cols, "prob_up", n_samples=900)
    display(figure)


#### Visualisation des scénarios FOMO

**Objectif** : comparer visuellement la convergence (ou divergence) entre les cotes simulées et la probabilité de marché pour chaque horizon (m15, h1, daily).

**Sorties attendues** : trois graphiques ligne par ligne. Chaque figure affiche `prob_up` (marché) et les colonnes `odds_*` sélectionnées. Une proximité visuelle indique que le scénario reproduit bien la dynamique observée ; un écart persistant signale qu’il faut retuner les paramètres FOMO.


## Modèle de cotes (régression)

**Objectif** : entraîner, pour chaque horizon, un modèle de régression qui prédit le mid price Up/Down ainsi que la probabilité implicite du marché à partir des features techniques.

**Sorties attendues** : un dictionnaire `regression_artifacts` contenant les pipelines scikit-learn et un tableau `regression_metrics` (MAE/RMSE par cible). Une faible MAE indique que le modèle reproduit correctement les cotes marché/FOMO.


In [ ]:
regression_artifacts = {}
regression_metrics = {}
for tf in timeframes:
    artifacts = train_odds_regressors(
        regression_data[tf]["dataset"],
        regression_data[tf]["features"],
        regression_data[tf]["targets"],
    )
    regression_artifacts[tf] = artifacts
    regression_metrics[tf] = artifacts.metrics

regression_metrics


#### Visualisation des cotes prédites

**Objectif** : comparer les sorties des régressions (`pred_*`) aux probabilités implicites de marché pour évaluer qualitativement la calibration.

**Sorties attendues** : trois graphiques (m15, h1, daily) affichant la cote de marché et les prédictions du modèle. Une superposition serrée valide la qualité du modèle ; de larges écarts signalent un besoin de features ou d’hyperparamètres supplémentaires.


In [ ]:
regression_predictions = {}
for tf in timeframes:
    dataset = regression_data[tf]["dataset"].copy()
    predictions = predict_regressions(regression_artifacts[tf], dataset)
    regression_predictions[tf] = predictions
    figure = plot_odds_comparison(
        predictions.assign(prob_up=predictions[f"{tf}_prob_up_market"]),
        tf,
        [f"pred_{target}" for target in regression_artifacts[tf].target_columns],
        "prob_up",
        n_samples=600,
    )
    display(figure)


## Modèle de probabilité réelle (classification)

**Objectif** : estimer la probabilité « réelle » de clôture Up pour chaque contrat, en exploitant l’historique de prix et les features techniques, indépendamment de la cote de marché instantanée.

**Sorties attendues** : un dictionnaire `classification_artifacts` et un tableau `classification_metrics` (AUC, accuracy, Brier). Une AUC > 0.5 indique une meilleure discrimination que le hasard, tandis qu’un Brier faible traduit une bonne calibration.

In [ ]:
classification_artifacts = {}
classification_metrics = {}
for tf in timeframes:
    artifacts = train_outcome_classifiers(
        classification_data[tf]["dataset"],
        classification_data[tf]["features"],
        {tf: classification_data[tf]["target"]},
    )
    classification_artifacts[tf] = artifacts
    classification_metrics[tf] = artifacts.metrics

classification_metrics


#### Diagnostic de calibration

**Objectif** : visualiser la calibration des probabilités prédites par les classifieurs sur l’échantillon de test.

**Sorties attendues** : trois courbes de calibration où la diagonale représente une calibration parfaite. Si les points sont au-dessus (ou en dessous), le modèle sous-estime (ou surestime) la probabilité de clôture Up.


In [ ]:
classification_predictions = {}
for tf in timeframes:
    dataset = classification_data[tf]["dataset"].copy()
    preds = predict_classifications(classification_artifacts[tf], dataset)
    classification_predictions[tf] = preds
    calibration_fig = plot_calibration_curve(
        preds,
        f"pred_proba_{tf}",
        classification_data[tf]["target"],
    )
    display(calibration_fig)


## Cotes synthétiques historiques et backtests

**Objectif** : combiner les probabilités prédites avec les scénarios FOMO pour générer des cotes synthétiques historiques, puis évaluer deux stratégies (2 % capital / 4 % parts) via un backtest.

**Sorties attendues** : dictionnaires `backtest_inputs` et `backtest_results` remplis pour chaque timeframe. Les résultats permettent d’analyser la rentabilité et la robustesse du modèle sur données historiques.

In [ ]:
backtest_inputs = {}
backtest_results = {}
scenario_selection = {}

for tf in timeframes:
    minute_df = minute_history[tf].copy()
    cls_dataset, minute_features, _ = build_classification_dataset(minute_df, tf)
    model = classification_artifacts[tf].models[tf]
    probabilities = model.predict_proba(cls_dataset[minute_features])[:, 1]
    indices = cls_dataset["original_index"].values
    minute_df.loc[indices, f"{tf}_prob_up_market"] = probabilities
    minute_df.loc[indices, "pred_prob_up"] = probabilities

    fomo_input_minute = make_fomo_input(minute_df, tf)
    fomo_simulated = simulate_fomo_odds(fomo_input_minute, fomo_scenarios[tf])
    scenario_col = [col for col in fomo_simulated.columns if col.startswith("odds_")][0]
    scenario_selection[tf] = scenario_col

    selected_rows = fomo_simulated.loc[indices].copy()
    price_frame = probabilities_to_prices(
        selected_rows[scenario_col],
        average_spreads[tf]["spread_up"],
        average_spreads[tf]["spread_down"],
    ).reset_index(drop=True)

    backtest_frame = pd.DataFrame(
        {
            "timestamp": minute_df.loc[indices, "timestamp"].values,
            "prediction": minute_df.loc[indices, "pred_prob_up"].values,
            "market_prob": selected_rows[scenario_col].values,
            "outcome": minute_df.loc[indices, f"{tf}_target_up"].values,
        }
    ).reset_index(drop=True)
    backtest_frame = pd.concat([backtest_frame, price_frame], axis=1)
    backtest_inputs[tf] = backtest_frame

    params = BacktestParams(
        timeframe=tf,
        prediction_col="prediction",
        market_prob_col="market_prob",
        outcome_col="outcome",
        price_up_col="price_up_ask",
        price_down_col="price_down_ask",
        threshold=0.05,
        initial_capital=1_000.0,
        capital_risk_fraction=0.02,
        share_fraction=0.04,
    )
    backtest_results[tf] = run_backtest(params, backtest_frame)

backtest_inputs


#### Synthèse tabulaire des backtests

**Objectif** : consolider, pour chaque horizon, le nombre de trades, le winrate, le PnL total et l’équity finale pour les deux stratégies.

**Sorties attendues** : un dictionnaire `backtest_summaries` où chaque `DataFrame` présente les colonnes `trades`, `winrate`, `total_pnl`, `final_equity`. Utilisez ces valeurs pour repérer les horizons les plus rentables ou instables.


In [ ]:
backtest_summaries = {tf: result.summary for tf, result in backtest_results.items()}
backtest_summaries


In [ ]:
for tf in timeframes:
    equity_fig = plot_equity_curves(backtest_results[tf])
    display(equity_fig)


In [ ]:
exported_regression = {
    tf: export_regression_artifacts(regression_artifacts[tf], prefix=f"odds_{tf}")
    for tf in timeframes
}
exported_classification = {
    tf: export_classification_artifacts(classification_artifacts[tf], prefix=f"outcome_{tf}")
    for tf in timeframes
}

exported_regression, exported_classification


## Synthèse rapide

- Les métriques des modèles de cotes et de probabilité sont disponibles dans `regression_metrics` et `classification_metrics`.
- Les backtests pour chaque horizon sont accessibles via `backtest_summaries` et les courbes d'équité.
- Les modèles exportés (régression & classification) sont sauvegardés dans `models/` avec un préfixe par timeframe, utilisables directement via `code.main.run_live_inference`.
